# Create and save localizations

In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.append('acdcpp/Automatic-Circuit-Discovery/')
sys.path.append('acdcpp/')
# from acdc import TLACDCExperiment
# from acdcpp.ACDCPPExperiment import ACDCPPExperiment

In [2]:
import os
import sys
import re

# import acdc
# from acdc.TLACDCExperiment import TLACDCExperiment
# from acdc.acdc_utils import TorchIndex, EdgeType
import numpy as np
import torch as t
from torch import Tensor
import einops
import itertools

from transformer_lens import HookedTransformer, ActivationCache

import tqdm.notebook as tqdm
import plotly
from rich import print as rprint
from rich.table import Table

from jaxtyping import Float, Bool
from typing import Callable, Tuple, Union, Dict, Optional

import torch
import pickle

from ACDCPPExperiment import ACDCPPExperiment
from cb_utils.mask_utils import get_masks_from_acdcpp_exp

device = t.device('cuda') if t.cuda.is_available() else t.device('mps:0')
print(f'Device: {device}')

Device: cuda


## ACDCPP/EAP

In [3]:
# set up pipeline from acdcpp to edge mask
model = HookedTransformer.from_pretrained(
    'gpt2-small',
    center_writing_weights=False,
    center_unembed=False,
    fold_ln=False,
    device=device,
)
model.set_use_hook_mlp_in(True)
model.set_use_split_qkv_input(True)
model.set_use_attn_result(True)

Loaded pretrained model gpt2-small into HookedTransformer


In [4]:
torch.cuda.max_memory_allocated(device=device) / 1e9

0.666538496

### IOI Localization

In [5]:
from tasks.ioi.IOITask import IOITask_old, IOITask
# ioi_task = IOITask(batch_size=5, tokenizer=model.tokenizer, device=device, prep_acdcpp=True, acdcpp_N=25)
ioi_task = IOITask(batch_size=5, tokenizer=model.tokenizer, device=device, prep_acdcpp=True, acdcpp_N=25, nb_templates=1, prompt_type="ABBA")
ioi_task.set_logit_diffs(model)

ioi_metric = ioi_task.get_acdcpp_metric()
def negative_abs_ioi_metric(logits: Float[Tensor, "batch seq_len d_vocab"]):
    return -abs(ioi_metric(logits))

with t.no_grad():
    clean_logits = model(ioi_task.clean_data.toks)
    corrupt_logits = model(ioi_task.corr_data.toks)
    clean_logit_diff = ioi_task.ave_logit_diff(clean_logits, ioi_task.clean_data).item()
    corrupt_logit_diff = ioi_task.ave_logit_diff(corrupt_logits, ioi_task.corr_data).item()
    print(f'Clean logit diff: {clean_logit_diff:.3f}, Corrupt logit diff: {corrupt_logit_diff:.3f}')

OpenAI API key not found, will not be able to run evaluations on HPSAQ Task
OpenAI API key not found, will not be able to run evaluations on HPSAQ Task
Clean logit diff: 3.040117025375366, Corrupted logit diff: 1.1900196075439453
Clean logit diff: 3.040, Corrupt logit diff: 1.190


In [6]:
from ACDCPPExperiment import ACDCPPExperiment
from cb_utils.mask_utils import get_masks_from_acdcpp_exp
THRESHOLDS = [0.05]#np.arange(0.005, 0.155, 0.005)
RUN_NAME = 'abs_edge'


acdcpp_exp = ACDCPPExperiment(
    model=model,
    clean_data=ioi_task.clean_data.toks,
    corr_data=ioi_task.corr_data.toks,
    acdc_metric=negative_abs_ioi_metric,
    acdcpp_metric=ioi_metric,
    thresholds=THRESHOLDS,
    run_name=RUN_NAME,
    verbose=False,
    attr_absolute_val=True,
    save_graphs_after=-100,
    pruning_mode='edge',
    no_pruned_nodes_attr=1,
    run_acdc=False,
    run_acdcpp=True,
)
# e=acdcpp_exp.setup_exp(0.0)

# pruned_heads, num_passes, acdcpp_pruned_attrs, acdc_pruned_attrs, edges_after_acdcpp, edges_after_acdc = acdcpp_exp.run()
acdcpp_nodes, acdcpp_edges, acdcpp_mask_dict, acdcpp_weight_mask_attn_dict, acdcpp_weight_mask_mlp_dict = get_masks_from_acdcpp_exp(acdcpp_exp, threshold=THRESHOLDS[0])

# with open(f"localizations/eap/ioi/exp_threshold={THRESHOLDS[0]}", "wb") as f:
#     pickle.dump(acdcpp_exp, f)

with open(f"localizations/eap/ioi/gpt2_threshold={THRESHOLDS[0]}.pkl", "wb") as f:
    pickle.dump((acdcpp_nodes, acdcpp_edges, acdcpp_mask_dict, acdcpp_weight_mask_attn_dict, acdcpp_weight_mask_mlp_dict), f)

  0%|          | 0/1 [00:00<?, ?it/s]WARNING:root:cache_all is deprecated and will eventually be removed, use add_caching_hooks or run_with_cache


self.current_node=TLACDCInterpNode(blocks.11.hook_resid_post, [:])


100%|██████████| 1/1 [00:07<00:00,  7.68s/it]

dict_keys([-1, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0, 12])


In [27]:
len(acdcpp_edges)

159

In [7]:
from cb_utils.transformer import DemoTransformer
from cb_utils.models import load_demo_gpt2, tokenizer
edge_masks = True
weight_masks_attn = False
weight_masks_mlp = False
train_base_weights = False

mask_dict_superset = acdcpp_mask_dict if edge_masks else None
weight_mask_attn_dict = acdcpp_weight_mask_attn_dict if weight_masks_attn else None
weight_mask_mlp_dict = acdcpp_weight_mask_mlp_dict if weight_masks_mlp else None
base_weight_attn_dict = acdcpp_weight_mask_attn_dict if train_base_weights else None
base_weight_mlp_dict = acdcpp_weight_mask_mlp_dict if train_base_weights else None
model = load_demo_gpt2(means=False, edge_masks=edge_masks, mask_dict_superset=mask_dict_superset, weight_masks_attn=weight_masks_attn, weight_masks_mlp=weight_masks_mlp, weight_mask_attn_dict=weight_mask_attn_dict, weight_mask_mlp_dict=weight_mask_mlp_dict, train_base_weights=train_base_weights, base_weight_attn_dict=base_weight_attn_dict, base_weight_mlp_dict=base_weight_mlp_dict)

Using device: cuda:0


In [12]:
mask_dict_superset

{'embed': tensor([]),
 'a0.0': tensor([1.]),
 'a0.1': tensor([1.]),
 'a0.2': tensor([1.]),
 'a0.3': tensor([1.]),
 'a0.4': tensor([1.]),
 'a0.5': tensor([1.]),
 'a0.6': tensor([0.]),
 'a0.7': tensor([1.]),
 'a0.8': tensor([1.]),
 'a0.9': tensor([0.]),
 'a0.10': tensor([0.]),
 'a0.11': tensor([1.]),
 'm0': tensor([0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1.]),
 'a1.0': tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'a1.1': tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'a1.2': tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'a1.3': tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'a1.4': tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'a1.5': tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'a1.6': tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'a1.7': tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.]),
 'a1.8': tensor([1., 1., 1., 1.

In [29]:
ioi_task.get_test_accuracy(model)

1.0

In [36]:
model.blocks[3].edge_mask_attentions

Parameter containing:
tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1

### Induction

In [5]:
from tasks.induction.InductionTask import InductionTask
ind_task = InductionTask(batch_size=5, tokenizer=model.tokenizer, prep_acdcpp=True, seq_len=15, acdcpp_metric="ave_logit_diff")
ind_task.set_logit_diffs(model)
from acdcpp.ACDCPPExperiment import ACDCPPExperiment

ind_metric = ind_task.get_acdcpp_metric()
def negative_abs_ind_metric(logits: Float[Tensor, "batch seq_len d_vocab"]):
    return -abs(ind_metric(logits))

THRESHOLDS = [0.03]#np.arange(0.005, 0.155, 0.005)
RUN_NAME = 'abs_edge'

acdcpp_exp = ACDCPPExperiment(
    model=model,
    clean_data=ind_task.clean_data,
    corr_data=ind_task.corr_data,
    acdc_metric=negative_abs_ind_metric,
    acdcpp_metric=ind_metric,
    thresholds=THRESHOLDS,
    run_name=RUN_NAME,
    verbose=False,
    attr_absolute_val=True,
    save_graphs_after=-100,
    pruning_mode='edge',
    no_pruned_nodes_attr=1,
    run_acdc=False,
    run_acdcpp=True,
)

acdcpp_nodes, acdcpp_edges, acdcpp_mask_dict, acdcpp_weight_mask_attn_dict, acdcpp_weight_mask_mlp_dict = get_masks_from_acdcpp_exp(acdcpp_exp, threshold=THRESHOLDS[0])

# with open(f"localizations/eap/induction/exp_threshold={THRESHOLDS[0]}", "wb") as f:
    # pickle.dump(acdcpp_exp, f)

with open(f"localizations/eap/induction/gpt2_threshold={THRESHOLDS[0]}.pkl", "wb") as f:
    pickle.dump((acdcpp_nodes, acdcpp_edges, acdcpp_mask_dict, acdcpp_weight_mask_attn_dict, acdcpp_weight_mask_mlp_dict), f)

OpenAI API key not found, will not be able to run evaluations on HPSAQ Task
OpenAI API key not found, will not be able to run evaluations on HPSAQ Task


  0%|          | 0/1 [00:00<?, ?it/s]WARNING:root:cache_all is deprecated and will eventually be removed, use add_caching_hooks or run_with_cache


self.current_node=TLACDCInterpNode(blocks.11.hook_resid_post, [:])


100%|██████████| 1/1 [00:09<00:00,  9.07s/it]


dict_keys([-1, 11, 10, 9, 8, 7, 6, 5, 0, 1, 2, 3, 4, 12])


In [6]:
from cb_utils.transformer import DemoTransformer
from cb_utils.models import load_demo_gpt2, tokenizer
edge_masks = True
weight_masks_attn = False
weight_masks_mlp = False
train_base_weights = False

mask_dict_superset = acdcpp_mask_dict if edge_masks else None
weight_mask_attn_dict = acdcpp_weight_mask_attn_dict if weight_masks_attn else None
weight_mask_mlp_dict = acdcpp_weight_mask_mlp_dict if weight_masks_mlp else None
base_weight_attn_dict = acdcpp_weight_mask_attn_dict if train_base_weights else None
base_weight_mlp_dict = acdcpp_weight_mask_mlp_dict if train_base_weights else None
model = load_demo_gpt2(means=False, edge_masks=edge_masks, mask_dict_superset=mask_dict_superset, weight_masks_attn=weight_masks_attn, weight_masks_mlp=weight_masks_mlp, weight_mask_attn_dict=weight_mask_attn_dict, weight_mask_mlp_dict=weight_mask_mlp_dict, train_base_weights=train_base_weights, base_weight_attn_dict=base_weight_attn_dict, base_weight_mlp_dict=base_weight_mlp_dict)

Using device: cuda:0


In [11]:
induction_task = InductionTask(batch_size=5, tokenizer=tokenizer, prep_acdcpp=True, seq_len=15, acdcpp_metric="ave_logit_diff")
induction_task.get_test_accuracy(model)

1.0

In [17]:
# load masks/induction_nonuniform/edge_masks_localize=acdcpp/ckpts/mask_params_final.pth, as state_dict
model.load_state_dict(torch.load('masks/induction_nonuniform/edge_masks_localize=acdcpp/ckpts/mask_params_final.pth'))
induction_task.get_test_accuracy(model)

1.0

### Sports

In [18]:
threshold = 0.0005
import pickle
with open('localizations/eap/eap_sports/1000_graph.pkl', 'rb') as f:
    graph = pickle.load(f)

eap_edges = graph.top_edges(n=len(graph.eap_scores.flatten()), threshold=threshold)
# eap_edges = set()
# for i in range(eap_scores.shape[0]):
#     for j in range(eap_scores.shape[1]):
#         if eap_scores[i, j] > threshold:
            # eap_edges.add((get_node_name(graph.node_names[i], show_full_index=False), get_node_name(graph.node_names[j, show_full_index=False))))
print(graph.eap_scores)

tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  3.0412e-08,
          3.7267e-10,  8.7028e-07],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  3.3888e-10,
          3.1195e-12, -5.1668e-08],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ..., -9.8981e-08,
         -4.1402e-10,  6.5371e-07],
        ...,
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00, -5.1173e-07],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00, -7.1097e-09],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00]])


In [19]:
# get mean, 95th percentile, 99th percentile
eap_scores_np = graph.eap_scores.flatten().cpu().numpy()
print(np.mean(eap_scores_np))
print(np.percentile(eap_scores_np, 95))
print(np.percentile(eap_scores_np, 99))
print(np.percentile(eap_scores_np, 99.99))
# stdev
print(np.std(eap_scores_np))
print(len(eap_scores_np))
print(len(eap_edges))

4.6151484e-08
8.446967427744294e-07
9.18949547667582e-06
0.0005713927092902466
2.3046277e-05
3277824
752


In [21]:
"""
Convert format:
want: {((3, 'm3'), (3, 'a3.0')),
 ((4, 'm4'), (0, 'm0')),
 ((4, 'm4'), (3, 'a3.0')),
 ((5, 'a5.9'), (0, 'm0')),}

have:
[('mlp.0', 'mlp.2', 0.005653967149555683),
('head.0.14', 'mlp.0', 0.006188404746353626),]
...
"""
from cb_utils.mask_utils import get_formatted_edges_from_eap, get_masks_from_eap_exp
# formatted_eap_edges = get_formatted_edges_from_eap(eap_edges)
# formatted_eap_edges
with open('localizations/eap/eap_sports/1000_graph.pkl', 'rb') as f:
    graph = pickle.load(f)
acdcpp_nodes, acdcpp_edges, acdcpp_mask_dict, acdcpp_weight_mask_attn_dict, acdcpp_weight_mask_mlp_dict = get_masks_from_eap_exp(graph, threshold=0.0005, num_layers=32, num_heads=32, filter_neox=True)

Old len: 673
New len: 626


In [ ]:
with open(f"localizations/eap/eap_sports/pythia-2.8b_{threshold=}.pkl", "wb") as f:
    pickle.dump((acdcpp_nodes, acdcpp_edges, acdcpp_mask_dict, acdcpp_weight_mask_attn_dict, acdcpp_weight_mask_mlp_dict), f)

In [21]:
# for every value in acdcpp_mask_dict, print number of 0s in tensor
num_zeros = 0
for k, v in acdcpp_mask_dict.items():
    num_zeros += (v == 0).sum().item()
print(num_zeros)

220


In [27]:
acdcpp_mask_dict['a0.1']

tensor([1.])

In [47]:
from tasks import InductionTask
ind_task = InductionTask(batch_size=16, tokenizer=model.tokenizer, prep_acdcpp=True, seq_len=10, acdcpp_metric="ave_logit_diff")
ind_task.set_logit_diffs(model)
print(ind_task.clean_logit_diff, ind_task.corrupted_logit_diff)

12.292320251464844 1.4027974605560303


In [48]:
ind_metric = ind_task.get_acdcpp_metric()
def negative_abs_ind_metric(logits: Float[Tensor, "batch seq_len d_vocab"]):
    return -abs(ind_metric(logits))

with t.no_grad():
    clean_logits = model(ind_task.clean_data.cuda())
    corrupt_logits = model(ind_task.corr_data.cuda())
    clean_logit_diff = ind_task.ave_logit_diff(clean_logits, ind_task.clean_data).item()
    corrupt_logit_diff = ind_task.ave_logit_diff(corrupt_logits, ind_task.corr_data).item()
    
print(ind_metric(clean_logits))
print(ind_metric(corrupt_logits))

In [51]:
from ACDCPPExperiment import ACDCPPExperiment
from cb_utils.mask_utils import get_masks_from_acdcpp_exp
THRESHOLDS = [0.05]#np.arange(0.005, 0.155, 0.005)
RUN_NAME = 'abs_edge'

acdcpp_exp = ACDCPPExperiment(
    model=model,
    clean_data=ind_task.clean_data,
    corr_data=ind_task.corr_data,
    acdc_metric=negative_abs_ind_metric,
    acdcpp_metric=ind_metric,
    thresholds=THRESHOLDS,
    run_name=RUN_NAME,
    verbose=False,
    attr_absolute_val=True,
    save_graphs_after=-100,
    pruning_mode='edge',
    no_pruned_nodes_attr=1,
    run_acdc=False,
    run_acdcpp=True,
)
# e=acdcpp_exp.setup_exp(0.0)

# pruned_heads, num_passes, acdcpp_pruned_attrs, acdc_pruned_attrs, edges_after_acdcpp, edges_after_acdc = acdcpp_exp.run()
acdcpp_nodes, acdcpp_edges, acdcpp_mask_dict, acdcpp_weight_mask_attn_dict, acdcpp_weight_mask_mlp_dict = get_masks_from_acdcpp_exp(acdcpp_exp, threshold=THRESHOLDS[0])

  0%|          | 0/1 [00:00<?, ?it/s]WARNING:root:cache_all is deprecated and will eventually be removed, use add_caching_hooks or run_with_cache


self.current_node=TLACDCInterpNode(blocks.11.hook_resid_post, [:])


100%|██████████| 1/1 [00:08<00:00,  8.35s/it]

dict_keys([-1, 11, 10, 9, 8, 7, 5, 0, 1, 2, 3, 4, 6, 12])


## Causal Tracing

In [3]:
# set up pipeline from acdcpp to edge mask
model = HookedTransformer.from_pretrained(
    'gpt2-small',
    center_writing_weights=False,
    center_unembed=False,
    fold_ln=False,
    device=device,
)
model.set_use_hook_mlp_in(True)
model.set_use_split_qkv_input(True)
model.set_use_attn_result(True)

NameError: name 'device' is not defined

In [3]:
with open(f"localizations/causal_tracing/induction/gpt2_small_attrs.pkl", "rb") as f:
    ct_result = pickle.load(f)

In [7]:
ct_result
""" looks like
{'a0.0': -0.0816584974527359,
 'a0.1': -0.10201289504766464,
 'a0.2': -0.12293148040771484,
 'a0.3': -0.04229861870408058,
 'a0.4': -0.13950654864311218,
 'a0.5': -0.08779175579547882,
 'a0.6': -0.10940369218587875,
 'a0.7': -0.12449425458908081,
 'a0.8': -0.1388336569070816,
 'a0.9': -0.07678628712892532,
 'a0.10': -0.043221551924943924,
 'a0.11': -0.10288099199533463,
 'm0': -0.09212236106395721,
 ...
 }
 """
# get the top 20% of the values
import numpy as np
ct_values = np.array(list(ct_result.values()))
threshold = np.percentile(ct_values, 80)

# get the keys that are above the threshold
ct_keys = list(ct_result.keys())
ct_keys_above_threshold = [k for k in ct_keys if ct_result[k] > threshold]
ct_keys_above_threshold

['a9.0',
 'a9.2',
 'a9.3',
 'a9.7',
 'a9.8',
 'a9.10',
 'a9.11',
 'a10.0',
 'a10.1',
 'a10.2',
 'a10.3',
 'a10.4',
 'a10.5',
 'a10.6',
 'a10.7',
 'a10.8',
 'a10.9',
 'a10.10',
 'a10.11',
 'a11.0',
 'a11.1',
 'a11.2',
 'a11.3',
 'a11.4',
 'a11.5',
 'a11.6',
 'a11.7',
 'a11.8',
 'a11.9',
 'a11.10',
 'a11.11']

In [15]:
from cb_utils.mask_utils import get_masks_from_ct_nodes

nodes_set, edges_set, ct_mask_dict, ct_weight_mask_attn_dict, ct_weight_mask_mlp_dict = get_masks_from_ct_nodes(ct_keys_above_threshold)
with open(f"localizations/causal_tracing/induction/gpt2_small_localizations.pkl", "wb") as f:
    pickle.dump((nodes_set, edges_set, ct_mask_dict, ct_weight_mask_attn_dict, ct_weight_mask_mlp_dict), f)

## Induction

In [11]:
from localizations.causal_tracing.causal_tracing import causal_tracing_induction
from tasks import InductionTask


from localizations.causal_tracing.causal_tracing import get_causal_tracing_components

ind_task_temp = InductionTask(batch_size=16, tokenizer=model.tokenizer)
ind_task = torch.stack(
    ind_task_temp.train_data,
    dim=0
)
components = causal_tracing_induction(
    model,
    ind_task
)

AttributeError: 'InductionTask' object has no attribute 'train_data'

## IOI

In [7]:
from localizations.causal_tracing.causal_tracing import causal_tracing_ioi 
from tasks.ioi.IOITask import IOITask_old, IOITask

ioi_task = IOITask(batch_size=5, tokenizer=model.tokenizer, device=device).ioi_data

components = causal_tracing_ioi(
    model,
    ioi_task
)

with open(f"localizations/causal_tracing/ioi/gpt2_threshold=all.pkl", "wb") as f:
    pickle.dump(components, f)

  0%|          | 0/100 [00:00<?, ?it/s]

Layer 1 MLP, Score 0.7341734170913696
Layer 2 MLP, Score 0.7266609072685242
Layer 9 MLP, Score 0.718209981918335
Layer 5 MLP, Score 0.6950880289077759
Layer 10 MLP, Score 0.6915597319602966
Layer 3 MLP, Score 0.6897463798522949
Layer 0 MLP, Score 0.6831357479095459
Layer 4 MLP, Score 0.677943229675293
Layer 7 MLP, Score 0.6773182153701782
Layer 6 MLP, Score 0.6743562817573547
Layer 11 MLP, Score 0.6734477877616882
Layer 8 MLP, Score 0.672644853591919
Layer 1 Attention Head 0, Score 0.0
Layer 2 Attention Head 1, Score 0.0
Layer 1 Attention Head 1, Score 0.0
Layer 1 Attention Head 2, Score 0.0
Layer 1 Attention Head 3, Score 0.0
Layer 1 Attention Head 4, Score 0.0
Layer 1 Attention Head 5, Score 0.0
Layer 1 Attention Head 6, Score 0.0
Layer 1 Attention Head 7, Score 0.0
Layer 1 Attention Head 8, Score 0.0
Layer 1 Attention Head 9, Score 0.0
Layer 1 Attention Head 10, Score 0.0
Layer 1 Attention Head 11, Score 0.0
Layer 2 Attention Head 0, Score 0.0
Layer 11 Attention Head 11, Score 0.0
L

TypeError: dump() missing required argument 'file' (pos 2)

In [10]:
### LOAD TASKS
from tasks.induction.InductionTask import InductionTask
from tasks.ioi.IOITask import IOITask
# from tasks.facts.SportsTask import SportsFactsTask

# ind_task = InductionTask(batch_size=5, tokenizer=tokenizer, device=device)
# ioi_task = IOITask(batch_size=5, tokenizer=tokenizer, device=device, prep_acdcpp=True)
ioi_task = IOITask(batch_size=5, tokenizer=model.tokenizer, device=device, prep_acdcpp=True, acdcpp_N=25, nb_templates=1, prompt_type="ABBA")
ioi_task.set_logit_diffs(model)
# sports_task = SportsFactsTask(model, N=5, batch_size=5, tokenizer=tokenizer, device=device)

#%%
from localizations.causal_tracing.causal_tracing import causal_tracing_ioi
from cb_utils.mask_utils import get_masks_from_ct_nodes
import numpy as np
import pickle

result = causal_tracing_ioi(model, ioi_task)
ct_keys = list(result.keys())
# threshold = 0.005
percentile = 80
threshold = np.percentile(list(result.values()), percentile)
ct_keys_above_threshold = [k for k in ct_keys if result[k] > threshold]
print(f"{threshold=}, {len(ct_keys)=}, {len(ct_keys_above_threshold)=}")

nodes_set,edges_set, ct_mask_dict, ct_weight_mask_attn_dict, ct_weight_mask_mlp_dict = get_masks_from_ct_nodes(ct_keys_above_threshold)

with open(f"localizations/causal_tracing/ioi/gpt2_{percentile=}.pkl", "wb") as f:
    pickle.dump((nodes_set, edges_set, ct_mask_dict, ct_weight_mask_attn_dict, ct_weight_mask_mlp_dict), f)

145

## Loading models

In [58]:
import pickle
from cb_utils.transformer import DemoTransformer
from cb_utils.models import load_demo_gpt2, tokenizer
means_ioi = True
if means_ioi:
    with open("data/gpt2_ioi_abc_means.pkl", "rb") as f:
        means = pickle.load(f)[0]
else:
    with open("data/gpt2_means.pkl", "rb") as f:
        means = pickle.load(f)[0]

edge_masks = True
weight_masks_attn = True
weight_masks_mlp = True
train_base_weights = True
localize_acdcpp = True

# if edge_masks is True, then have mask_dict_superset be acdcpp_mask_dict
mask_dict_superset = None if not edge_masks else acdcpp_mask_dict
# model = load_demo_gpt2(means=means, mask_dict_superset=acdcpp_mask_dict)
if localize_acdcpp:
    weight_mask_attn_dict = acdcpp_weight_mask_attn_dict if weight_masks_attn else None
    weight_mask_mlp_dict = acdcpp_weight_mask_mlp_dict if weight_masks_mlp else None
    base_weight_attn_dict = acdcpp_weight_mask_attn_dict if train_base_weights else None
    base_weight_mlp_dict = acdcpp_weight_mask_mlp_dict if train_base_weights else None

else:
    weight_mask_attn_dict = None
    weight_mask_mlp_dict = None
    base_weight_attn_dict = None
    base_weight_mlp_dict = None

# model = load_demo_gpt2(means=False, edge_masks=edge_masks, mask_dict_superset=mask_dict_superset, weight_masks_attn=weight_masks_attn, weight_masks_mlp=weight_masks_mlp, weight_mask_attn_dict=weight_mask_attn_dict, weight_mask_mlp_dict=weight_mask_mlp_dict, train_base_weights=train_base_weights, base_weight_attn_dict=base_weight_attn_dict, base_weight_mlp_dict=base_weight_mlp_dict)

Using device: cuda:0


In [7]:
from cb_utils.models import load_demo_pythia
model = load_demo_pythia(means=False, )

Using device: cuda:0
Loaded pretrained model pythia-2.8b into HookedTransformer


In [8]:
reference_pythia =  HookedTransformer.from_pretrained(
        'pythia-2.8b',
        fold_ln=False,
        center_writing_weights=False,
        center_unembed=False,
        default_padding_side="left",
        device='cuda'
    )
tokenizer = reference_pythia.tokenizer

Loaded pretrained model pythia-2.8b into HookedTransformer


In [9]:
pythia_tokenizer = reference_pythia.tokenizer

In [14]:
# compare model outputs
test_input = t.tensor(pythia_tokenizer.encode("The quick brown fox jumps over the lazy")).unsqueeze(0).cuda()
assert (model(test_input)[0][0, -1] - reference_pythia(test_input)[0, -1]).abs().max() < 1e-5, (model(test_input)[0][0, -1] - reference_pythia(test_input)[0, -1]).abs().max()

AssertionError: tensor(1.8776, device='cuda:0', grad_fn=<MaxBackward1>)

In [15]:
model(test_input)[0][0, -1]

tensor([ 8.5317, -2.0861,  9.2626,  ..., -2.3424, -2.3985, -2.5804],
       device='cuda:0', grad_fn=<SelectBackward0>)

In [17]:
reference_pythia(test_input)[0, -1]

tensor([ 6.6789, -3.9404,  7.4135,  ..., -4.1955, -4.2537, -4.4353],
       device='cuda:0', grad_fn=<SelectBackward0>)

In [11]:
pythia_tokenizer.decode(torch.argmax(model(test_input)[0][0, -1]))

' dog'

In [12]:
pythia_tokenizer.decode(torch.argmax(reference_pythia(test_input)[0, -1]))

' dog'

In [31]:
from tasks.induction.InductionTask import InductionTask, InductionTask_Uniform

ind_uniform_task = InductionTask_Uniform(batch_size=16, tokenizer=tokenizer, prep_acdcpp=False, seq_len=15, uniform_over="rep_tokens")
ind_uniform_task.get_test_loss(model)

tensor(16., device='cuda:0')


tensor(14.1681, device='cuda:0')

In [25]:
ind_task.get_test_loss(model)

tensor(0.6161, device='cuda:0')

In [64]:
torch.cuda.memory_allocated(device=device) / 1e9

16.789679616

## Test that gradients flow correctly

In [8]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.shape, param.requires_grad)
    # print(name, param.shape, param.requires_grad)

output_mask torch.Size([157]) True
blocks.0.edge_mask_attentions torch.Size([1, 12]) True
blocks.0.edge_mask_mlp torch.Size([13]) True
blocks.1.edge_mask_attentions torch.Size([14, 12]) True
blocks.1.edge_mask_mlp torch.Size([26]) True
blocks.2.edge_mask_attentions torch.Size([27, 12]) True
blocks.2.edge_mask_mlp torch.Size([39]) True
blocks.3.edge_mask_attentions torch.Size([40, 12]) True
blocks.3.edge_mask_mlp torch.Size([52]) True
blocks.4.edge_mask_attentions torch.Size([53, 12]) True
blocks.4.edge_mask_mlp torch.Size([65]) True
blocks.5.edge_mask_attentions torch.Size([66, 12]) True
blocks.5.edge_mask_mlp torch.Size([78]) True
blocks.6.edge_mask_attentions torch.Size([79, 12]) True
blocks.6.edge_mask_mlp torch.Size([91]) True
blocks.7.edge_mask_attentions torch.Size([92, 12]) True
blocks.7.edge_mask_mlp torch.Size([104]) True
blocks.8.edge_mask_attentions torch.Size([105, 12]) True
blocks.8.edge_mask_mlp torch.Size([117]) True
blocks.9.edge_mask_attentions torch.Size([118, 12]) Tr

In [10]:
from tasks import IOITask
model.train()
batch_size = 8
ioi = IOITask(batch_size=batch_size, tokenizer=tokenizer, device=device, prep_acdcpp=False)
loss = ioi.get_train_loss(model)
loss.backward()

In [11]:
losses = []
for i in range(10):
    losses.append(ioi.get_train_loss(model))
    print(torch.cuda.memory_allocated(device=device) / 1e9)

3.556975616
4.58348544
5.60505856
6.62846464
7.652282368
8.674494464
9.697204224
10.718908416
11.741693952
12.768288768


In [11]:
param_names = []
model_params = []
for name, param in model.named_parameters():
    if param.requires_grad: # and "edge" in name:
        # check if param.grad is all zeros
        if param.grad is not None and param.grad.sum() != 0:
            print(f"{name} grad is not all zeros, {param.grad.norm()=}")
        else:
            print(f"{name} grad is all zeros")

output_mask grad is not all zeros, param.grad.norm()=tensor(2.0475, device='cuda:0')
blocks.0.edge_mask_attentions grad is not all zeros, param.grad.norm()=tensor(9.0644e-05, device='cuda:0')
blocks.0.edge_mask_mlp grad is not all zeros, param.grad.norm()=tensor(1.0276, device='cuda:0')
blocks.1.edge_mask_attentions grad is not all zeros, param.grad.norm()=tensor(0.0494, device='cuda:0')
blocks.1.edge_mask_mlp grad is not all zeros, param.grad.norm()=tensor(0.0643, device='cuda:0')
blocks.2.edge_mask_attentions grad is all zeros
blocks.2.edge_mask_mlp grad is all zeros
blocks.3.edge_mask_attentions grad is not all zeros, param.grad.norm()=tensor(0.0193, device='cuda:0')
blocks.3.edge_mask_mlp grad is not all zeros, param.grad.norm()=tensor(0.1006, device='cuda:0')
blocks.4.edge_mask_attentions grad is all zeros
blocks.4.edge_mask_mlp grad is not all zeros, param.grad.norm()=tensor(0.2149, device='cuda:0')
blocks.5.edge_mask_attentions grad is not all zeros, param.grad.norm()=tensor(0.0

### Check if correct MLPs flow

In [29]:
for node in acdcpp_nodes:
    if "m" in node[1]:
        print(node)

(2, 'm2')
(10, 'm10')
(5, 'm5')
(3, 'm3')
(7, 'm7')
(1, 'm1')
(9, 'm9')
(8, 'm8')
(4, 'm4')
(0, 'm0')
(-1, 'embed')
(6, 'm6')


In [31]:
param_names = []
model_params = []
for name, param in model.named_parameters():
    if "mlp" in name:
        # check if param.grad is all zeros
        if param.grad is not None and param.grad.sum() != 0:
            print(f"{name} grad is not all zeros, {param.grad.norm()=}")
        else:
            print(f"{name} grad is all zeros")

blocks.0.edge_mask_mlp grad is all zeros
blocks.0.edge_mask_mlp_baseline grad is all zeros
blocks.0.edge_mask_mlp_frozen grad is all zeros
blocks.0.mlp.W_in grad is not all zeros, param.grad.norm()=tensor(1.5009, device='cuda:0')
blocks.0.mlp.b_in grad is not all zeros, param.grad.norm()=tensor(0.3840, device='cuda:0')
blocks.0.mlp.W_out grad is not all zeros, param.grad.norm()=tensor(1.9580, device='cuda:0')
blocks.0.mlp.b_out grad is not all zeros, param.grad.norm()=tensor(0.2079, device='cuda:0')
blocks.1.edge_mask_mlp grad is all zeros
blocks.1.edge_mask_mlp_baseline grad is all zeros
blocks.1.edge_mask_mlp_frozen grad is all zeros
blocks.1.mlp.W_in grad is not all zeros, param.grad.norm()=tensor(2.1881, device='cuda:0')
blocks.1.mlp.b_in grad is not all zeros, param.grad.norm()=tensor(0.4780, device='cuda:0')
blocks.1.mlp.W_out grad is not all zeros, param.grad.norm()=tensor(2.0281, device='cuda:0')
blocks.1.mlp.b_out grad is not all zeros, param.grad.norm()=tensor(0.2563, device=

### Check if individual attention heads have gradients

In [76]:
for node in acdcpp_nodes:
    if "a9" in node[1]:
        print(node)

(9, 'a9.7')
(9, 'a9.9')
(9, 'a9.6')
(9, 'a9.8')


In [81]:
param_names = []
model_params = []
for name, param in model.named_parameters():
    if param.requires_grad and "9.attn.weight" in name:
    # if param.requires_grad and "9.attn.b" in name:
        print(param.shape)
        # if param.grad is not None and param.grad.sum() != 0:
        #     print(f"{name} grad is not all zeros, {param.grad.norm()=}")
        # else:
        #     print(f"{name} grad is all zeros")
        print(f"{param.grad[3].norm()=}")
        print(f"{param.grad[4].norm()=}")
        print(f"{param.grad[7].norm()=}")
        print(f"{param.grad[8].norm()=}")
        print(f"{param.grad[11].norm()=}")
        print()


torch.Size([12, 768, 64])
param.grad[3].norm()=tensor(0., device='cuda:0')
param.grad[4].norm()=tensor(0., device='cuda:0')
param.grad[7].norm()=tensor(0.1051, device='cuda:0')
param.grad[8].norm()=tensor(0.0287, device='cuda:0')
param.grad[11].norm()=tensor(0., device='cuda:0')

torch.Size([12, 768, 64])
param.grad[3].norm()=tensor(0., device='cuda:0')
param.grad[4].norm()=tensor(0., device='cuda:0')
param.grad[7].norm()=tensor(0.1273, device='cuda:0')
param.grad[8].norm()=tensor(0.0335, device='cuda:0')
param.grad[11].norm()=tensor(0., device='cuda:0')

torch.Size([12, 768, 64])
param.grad[3].norm()=tensor(0., device='cuda:0')
param.grad[4].norm()=tensor(0., device='cuda:0')
param.grad[7].norm()=tensor(0.0570, device='cuda:0')
param.grad[8].norm()=tensor(0.0485, device='cuda:0')
param.grad[11].norm()=tensor(0., device='cuda:0')

torch.Size([12, 64, 768])
param.grad[3].norm()=tensor(0., device='cuda:0')
param.grad[4].norm()=tensor(0., device='cuda:0')
param.grad[7].norm()=tensor(0.030

## Start Mask Learning

In [ ]:
from tasks import IOITask, SportsTask, OWTTask
batch_size = 64
ioi = IOITask(batch_size=batch_size, tokenizer=tokenizer, device=device, prep_acdcpp=False, prompt_type="ABBA", nb_templates=1, template_start_idx=0)
sports = SportsTask(batch_size=batch_size, tokenizer=tokenizer, device=device)
owt = OWTTask(batch_size=batch_size, tokenizer=tokenizer, device=device)

ioi_ood = IOITask(batch_size=batch_size, tokenizer=tokenizer, device=device, prep_acdcpp=False, prompt_type="ABBA", nb_templates=1, template_start_idx=1) # different template

train_tasks = {"ioi": ioi, "owt": owt}
task_weights = {"ioi": -.2, "owt": 1} # I think means preserve OWT, corrupt IOI
eval_tasks = {"ioi": ioi, "sports": sports, "owt": owt}

/data/phillip_guo/miniconda3/envs/unlrn/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [ ]:
mask_params = []
param_names = []
for name, p in model.named_parameters():
    if p.requires_grad:
        param_names.append(name)
        mask_params.append(p)

In [ ]:
from cb_utils.learn_mask import train_masks

epochs_left = 500
steps_per_epoch = 10
lr = .05 # free
weight_decay = 0
evaluate_every = 1
discretize_every = 50 # 5 # free
threshold = 0.5
use_wandb = False
edge_mask_reg_strength = None
weight_mask_reg_strength = 10

wandb_config = {"edge_masks": edge_masks, "weight_masks_attn": weight_masks_attn, "weight_masks_mlp": weight_masks_mlp, "epochs": epochs_left, "steps_per_epoch": steps_per_epoch, "lr": lr, "weight_decay": weight_decay, "evaluate_every": evaluate_every, "discretize_every": discretize_every, "threshold": threshold, "edge_mask_reg_strength": edge_mask_reg_strength, "weight_mask_reg_strength": weight_mask_reg_strength}

optimizer = torch.optim.AdamW(mask_params, lr=lr, weight_decay=weight_decay)
train_masks(model, tasks=train_tasks, optimizer=optimizer, num_epochs=epochs_left, steps_per_epoch=steps_per_epoch,
            # param_names=param_names, mask_params=mask_params, 
            task_weights=task_weights, eval_tasks=eval_tasks, evaluate_every=evaluate_every, discretize_every=discretize_every, threshold=threshold, edge_mask_reg_strength=edge_mask_reg_strength, weight_mask_reg_strength=None, verbose=False, use_wandb=use_wandb, wandb_config=wandb_config)

wandb: Currently logged in as: philliphguo. Use `wandb login --relogin` to force relogin


  6%|▋         | 32/501 [10:48<2:38:21, 20.26s/it]


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/data/phillip_guo/miniconda3/envs/unlrn/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_93609/3197613862.py", line 15, in <module>
    train_masks(model, tasks=train_tasks, optimizer=optimizer, num_epochs=epochs_left, steps_per_epoch=steps_per_epoch,
  File "/data/phillip_guo/mechanistic-unlearning/cb_utils/learn_mask.py", line 178, in train_masks
  File "/data/phillip_guo/miniconda3/envs/unlrn/lib/python3.10/site-packages/torch/_tensor.py", line 492, in backward
    torch.autograd.backward(
  File "/data/phillip_guo/miniconda3/envs/unlrn/lib/python3.10/site-packages/torch/autograd/__init__.py", line 251, in backward
    Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call l

In [ ]:
import pickle
with open(f"masks/trained_mask_params_{epochs_left=}_{edge_mask_reg_strength=}.pkl", "wb") as f:
    pickle.dump(mask_params, f)

In [ ]:
for name, p in zip(param_names, mask_params):
    if p.requires_grad:
        # print(name, p)
        # count how many zeros in p
        print(torch.sum(p == 0))

tensor(8, device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')
tensor(2, device='cuda:0')
tensor(3, device='cuda:0')
tensor(1, device='cuda:0')
tensor(0, device='cuda:0')
tensor(1, device='cuda:0')
tensor(0, device='cuda:0')
tensor(1, device='cuda:0')
tensor(2, device='cuda:0')
tensor(0, device='cuda:0')
tensor(1, device='cuda:0')
tensor(0, device='cuda:0')
tensor(6, device='cuda:0')
tensor(0, device='cuda:0')
tensor(12, device='cuda:0')
tensor(0, device='cuda:0')
